In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [1]:
import pandas as pd

train = pd.read_parquet('/kaggle/input/nuclear-vk-bots/train.parquet')
test = pd.read_parquet('/kaggle/input/nuclear-vk-bots/test.parquet')
unsupervised = pd.read_parquet('/kaggle/input/vk-nuclear-unlabeled/unlabelled.snappy.parquet')
unsupervised = unsupervised[:500000]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
train['text'] = train.apply(lambda row: f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}", axis=1)
test['text'] = test.apply(lambda row: f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}", axis=1)
unsupervised['text'] = unsupervised.apply(lambda row: f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}", axis=1)

In [6]:
train_new = train.drop(["ciphers", "curves", "ua"], axis = 1)
test_new = test.drop(["ciphers", "curves", "ua"], axis = 1)
unsup_new = unsupervised.drop(["ciphers", "curves", "ua"], axis = 1)

In [7]:
from itertools import chain
tokenizer = tokenizer.train_new_from_iterator([f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}" for row in chain(train.itertuples(), test.itertuples())], 
                                                   vocab_size= 15000)

In [8]:
text = "0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 [MASK]"

In [9]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'>>> 0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 0xc050'
'>>> 0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 279'
'>>> 0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 0xc073'
'>>> 0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 '
'>>> 0xcaca ECDHE-RSA-AES128-SHA 0x1301 0x1303 0xc076'


In [10]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_new)
test_data = Dataset.from_pandas(test_new)
unsupervised_data = Dataset.from_pandas(unsup_new)

In [11]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_dataset = unsupervised_data.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_dataset

  0%|          | 0/500 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (868 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 500000
})

In [12]:
tokenizer.model_max_length

512

In [13]:
chunk_size = 128

In [14]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 193'
'>>> Review 1 length: 257'
'>>> Review 2 length: 209'


In [15]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 659'


In [16]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 19'


In [17]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_datasets = tokenized_dataset.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/500 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 855186
})

In [20]:
tokenizer.decode(lm_datasets[1]["input_ids"])

"( linux ; arm _ 64 ; android 10 ; sne - lx1 ) applewebkit / 537. 36 ( khtml, like gecko ) chrome / 102. 0. 5005. 148 yaapp _ android / 22. 73. 1 yasearchbrowser / 22. 73. 1 bropp / 1. 0 sa / 3 mobile safari / 537. 36 [SEP] [CLS] ['0x1303'' ecdhe - ecdsa - aes128 - gcm - sha256'' aes256 - gcm - sha384'' des - cbc3 - sha'' ecdhe - ecdsa - aes256 - sha'' ecdhe - rsa - des - cbc3 - sha'' 0x1302'' 0xcca9'' ecdhe - rsa - aes128 -"

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [23]:
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] ['aes256 - gcm scorpion sha384'' 0xcca8'' aes256 - sha'[MASK] ecdhe - ecdsa - aes256 - gcm [MASK] sha384'' 0x1301'' ecdhe - rsa - aes128 - gcm [MASK] sha256'' ecdhe - rsa - aes128 - sha'' 0xcca9 [MASK]'ecdhe - rsa - aes256 - [MASK]'' aes128 - sha'' aes128 - gcm - sha256'' 0x0a0a'' 0x1303'' ecdhe - ecdsa - aes128 - gcm - sha256'' [MASK]'' ecdhe - [MASK] [MASK] aes256 - gcm - sha384'] [SEP] ['0x001d'' prime256v1'' 12b436'' [MASK]'] [SEP] mozilla [MASK] 5. 0'

'>>> ( [MASK] ; arm _ 64 [MASK] android 10 ; sne [MASK] lx1 ) applewebkit / 537. 36 [MASK] khtml, like gecko ) [MASK] / 102. 0. 5005. 148 yaapp _ android / 22. 73. 1 yasearchbrowser [MASK] 22. 73. [MASK] [MASK] / 1. [MASK] sa / 3 mobile safari / 537. 36 [SEP] [CLS] [MASK] '145'' ecdhe - [MASK] - aes128 - gcm - t560nu'' aes256 - gcm 4654 sha384'' des - cbc3 - sha'' [MASK] - ecdsa - ale - sha'[MASK] ecdhe - rsa - des - cbc3 - sha'' 0x1302'' 0xcca9'' ecdhe [MASK] [MASK] - aes128 -'


In [22]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [25]:
samples = [lm_datasets[i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [ [MASK] aes256 - gcm - [MASK] [MASK]'0xcca8'' [MASK] [MASK] sha'' [MASK] - ecdsa - aes256 - gcm [MASK] [MASK]'' [MASK]'' ecdhe - rsa - aes128 - [MASK] - [MASK]'' ecdhe - rsa - aes128 - sha'' 0xcca9'' [MASK] - rsa - aes256 - [MASK]'' aes128 - sha'[MASK] aes128 - gcm - sha256 [MASK]'0x0a0a'' 0x1303'' [MASK] - ecdsa - aes128 - gcm - sha256'' 0x1302'' ecdhe - rsa - [MASK] - gcm [MASK] sha384 [MASK] ] [SEP] [ [MASK] 0x001d'' prime256v1'[MASK] secp384r1'' 0x3a3a'] [SEP] mozilla [MASK] 5. [MASK]'

'>>> [MASK] linux ; arm _ [MASK] ; android [MASK] ; sne - lx1 ) [MASK] / 537. 36 ( [MASK], [MASK] gecko ) chrome [MASK] 102. 0 [MASK] [MASK]. 148 yaapp _ android / 22. 73 [MASK] 1 yasearchbrowser [MASK] 22. [MASK]. 1 bropp / 1 [MASK] 0 sa / 3 [MASK] safari / [MASK]. 36 [SEP] [CLS] [ [MASK] 0x1303 [MASK]'ecdhe - [MASK] - aes128 [MASK] gcm [MASK] [MASK]'' aes256 [MASK] [MASK] - sha384'' des - cbc3 - sha'' ecdhe - ecdsa - aes256 [MASK] [MASK]'' ecdhe - rsa - des - [MASK] - sha'' 0x1302 [MA

In [27]:
train_size = 770_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 770000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 77000
    })
})

In [28]:
from huggingface_hub import notebook_login

notebook_login()

In [34]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-vk",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    num_train_epochs = 1
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/kaggle/working/distilbert-base-uncased-finetuned-vk is already a clone of https://huggingface.co/bruhwalkk/distilbert-base-uncased-finetuned-vk. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [31]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


>>> Perplexity: 5611962.26


In [32]:
del eval_results
del lm_datasets

In [36]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,0.321900,0.215033


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=12032, training_loss=0.3218488066675181, metrics={'train_runtime': 7405.1349, 'train_samples_per_second': 103.982, 'train_steps_per_second': 1.625, 'total_flos': 2.551802254848e+16, 'train_loss': 0.3218488066675181, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 11.32

In [37]:
trainer.push_to_hub()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

Upload file runs/Apr30_16-30-17_4bfea5b481af/events.out.tfevents.1682872232.4bfea5b481af.24.2:   0%|          …

To https://huggingface.co/bruhwalkk/distilbert-base-uncased-finetuned-vk
   f7f4add..dee30f5  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/bruhwalkk/distilbert-base-uncased-finetuned-vk
   dee30f5..b088aff  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/bruhwalkk/distilbert-base-uncased-finetuned-vk/commit/dee30f5d694e41ba1c11f29f94dc5ea3a470f807'

In [38]:
trainer.save_model('/kaggle/working/gorwok')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [40]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="bruhwalkk/distilbert-base-uncased-finetuned-vk"
)

In [41]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> 0xcaca ecdhe - rsa - aes128 - sha 0x1301 0x1303 ]
>>> 0xcaca ecdhe - rsa - aes128 - sha 0x1301 0x1303 '
>>> 0xcaca ecdhe - rsa - aes128 - sha 0x1301 0x1303 )
>>> 0xcaca ecdhe - rsa - aes128 - sha 0x1301 0x1303 secp384r1
>>> 0xcaca ecdhe - rsa - aes128 - sha 0x1301 0x1303 aes128
